In [142]:
#Importing the API and instantiating the REST client according to our keys
from alpaca.trading.client import TradingClient
from alpaca_trade_api.rest import REST, TimeFrame
from websocket import create_connection
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
import json 
import pprint 
import talib 

API_KEY = "PKBEMHNKX7XNX73CWYLW"
SECRET_KEY = "7kDqb81BBOGfxwQc4EMwDVczEJQaDRGq1MoCdrrb"
url = "wss://stream.data.alpaca.markets/v1beta1/crypto"

trading_client = TradingClient(API_KEY, SECRET_KEY, paper=True)

In [143]:
# BUY ORDER
market_order_buy = MarketOrderRequest(
                      symbol="BTC/USD",
                      qty=1,
                      side=OrderSide.BUY,
                      time_in_force=TimeInForce.GTC
                  )

#SELL ORDER 
market_order_sell = MarketOrderRequest(
                      symbol="BTC/USD",
                      qty=1,
                      side=OrderSide.SELL,
                      time_in_force=TimeInForce.GTC
                  )

In [145]:
api = REST(key_id=API_KEY, secret_key=SECRET_KEY, base_url=url)

# Fetch 1Minute historical bars of Bitcoin
data = api.get_crypto_bars("BTCUSD", TimeFrame.Minute).df
data = data[data.exchange =='CBSE']
data

,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2023-02-13 06:00:00+00:00,CBSE,21839.50,21845.05,21839.50,21841.75,1.437640,106,21842.317525
2023-02-13 06:01:00+00:00,CBSE,21847.20,21848.18,21840.44,21846.24,0.834262,88,21844.794199
2023-02-13 06:02:00+00:00,CBSE,21845.92,21845.98,21838.98,21839.43,0.627565,76,21842.383313
2023-02-13 06:03:00+00:00,CBSE,21839.42,21839.94,21831.27,21831.27,1.565863,104,21835.793740
2023-02-13 06:04:00+00:00,CBSE,21832.02,21835.17,21829.98,21832.93,1.841734,110,21832.642307
...,...,...,...,...,...,...,...,...
2023-02-13 13:23:00+00:00,CBSE,21649.00,21649.00,21629.89,21634.22,10.618296,233,21636.920579
2023-02-13 13:24:00+00:00,CBSE,21634.20,21646.67,21633.38,21636.45,6.427886,179,21640.651296
2023-02-13 13:25:00+00:00,CBSE,21636.77,21650.38,21635.21,21644.95,7.024873,162,21640.322282


In [146]:
#DEFINE THE MACD AND SELL/BUY SIGNALS
exp1 = data['close'].ewm(span=12).mean()
exp2 = data['close'].ewm(span=26).mean() 
macd = exp1 - exp2
signal = macd.ewm(span=9).mean()


msell = (signal >= 6)
mbuy = (signal <= -4)
mbuy

timestamp
2023-02-13 06:00:00+00:00    False
2023-02-13 06:01:00+00:00    False
2023-02-13 06:02:00+00:00    False
2023-02-13 06:03:00+00:00    False
2023-02-13 06:04:00+00:00    False
                             ...  
2023-02-13 13:23:00+00:00     True
2023-02-13 13:24:00+00:00     True
2023-02-13 13:25:00+00:00     True
2023-02-13 13:26:00+00:00     True
2023-02-13 13:27:00+00:00     True
Name: close, Length: 448, dtype: bool

In [151]:
#buying specified security
api = REST(key_id=API_KEY, secret_key=SECRET_KEY, base_url=url)

# Fetch 1Minute historical bars of Bitcoin
data = api.get_crypto_bars("BTCUSD", TimeFrame.Minute).df
data = data[data.exchange =='CBSE']

def MACD(self):
  if msell is True:
    market_order = trading_client.submit_order(market_order_sell)
  for property_name, value in market_order:
    print(f"\"{property_name}\": {value}")
  else:
   pass


  if mbuy is True:
    market_order = trading_client.submit_order(market_order_buy)
  for property_name, value in market_order:
    print(f"\"{property_name}\": {value}")
  else:
    pass

positions = trading_client.get_all_positions()
for position in positions:
    for property_name, value in position:
        print(f"\"{property_name}\": {value}")

position

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('64bbff51-59d6-4b3c-9351-13ad85e3c752'),
    'avg_entry_price': '21717.0771860566942781',
    'change_today': '0.017158327230481',
    'cost_basis': '86651.13782034666986716',
    'current_price': '22028.13',
    'exchange': <AssetExchange.FTXU: 'FTXU'>,
    'lastday_price': '21656.54',
    'market_value': '87892.23854580309',
    'qty': '3.989999993',
    'side': <PositionSide.LONG: 'long'>,
    'symbol': 'BTCUSD',
    'unrealized_intraday_pl': '1241.1007254564201327778599467',
    'unrealized_intraday_plpc': '0.0143229593595134',
    'unrealized_pl': '1241.10072545642013284',
    'unrealized_plpc': '0.0143229593595134'}

In [ ]:
links = https://analyzingalpha.com/python-trading-tools

In [150]:
positions = trading_client.get_all_positions()
for position in positions:
    for property_name, value in position:
        print(f"\"{property_name}\": {value}")

In [79]:
from alpaca.trading.requests import GetAssetsRequest

# Check our current balance vs. our balance at the last market close
balance_change = float(account.equity) - float(account.last_equity)
print(f'Today\'s portfolio balance change: ${balance_change}')

Today's portfolio balance change: $-187.9278273186792
